In [1]:
#Define an n x n grid size around your tropical wave to extract data from (we'll average the values) 
# 3. Set window size (number of grid points on each side of center)
half_n = 2  # For a 5x5 box → 2 on each side + center


In [2]:
# Import relevant packages


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import random
import nbformat
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
import csv

2025-05-08 17:14:23.430852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

# # Load the notebook
# with open("post_process_tc_wave_cckw_interactions_automated.ipynb") as f:
#     nb = nbformat.read(f, as_version=4)

# # Get the code from the cells
# code_cells = [cell['source'] for cell in nb.cells if cell.cell_type == 'code']

# # Execute a specific cell (e.g., cell 3) to get the variable
# exec(code_cells[0])  # Be careful: this executes code!

# # Now the variable from that cell will be in your namespace
# print(dt_cckw_cross_hrs)  # Assuming cell 3 defines `my_variable`

In [4]:
merged_cckw = pd.read_csv('merged_cckw.csv')
merged_cckw

,ID,fhr,time,lon,lat,vor850,day_adj,cckw_crest_cross,tc_genesis,developer
0,139,138.0,2000-04-04 18:00:00,278.625671,8.888637,3.522798,6.25,0.0,0.0,1.0
1,413,186.0,2000-04-06 18:00:00,198.612503,6.923033,2.540011,8.25,0.0,0.0,1.0
2,774,318.0,2000-04-12 06:00:00,162.555420,11.464490,4.961282,13.75,0.0,0.0,1.0
3,907,522.0,2000-04-20 18:00:00,237.747238,9.135058,3.187877,22.25,0.0,0.0,1.0
4,1060,498.0,2000-04-19 18:00:00,47.579945,7.618085,4.137427,21.25,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
223,6073,2844.0,2000-07-26 12:00:00,265.664886,19.568724,2.247950,119.00,0.0,0.0,0.0
224,6113,2760.0,2000-07-23 00:00:00,133.374588,14.434371,0.718992,115.50,0.0,0.0,0.0
225,6279,3018.0,2000-08-02 18:00:00,86.448380,4.810503,2.179097,126.25,0.0,0.0,0.0
226,6302,3084.0,2000-08-05 12:00:00,358.855286,5.776806,1.209360,129.00,0.0,0.0,0.0


In [5]:
#Now let's add key variable anomalies that may be predictive of tc genesis at the track's center. Let's make sure we do this systematically! 

#Key variables (for now)
key_vars ={'olrtoa','mse_vint','u200','v200','u850','v850','qv_vint'}

#What I want to do: 
#1) For each row in merged_cckw, find nearest lat-lon-time data point 
#2) Find the value of the key variables at that point in space and time 
#3) Anomalies must be created - we can do this by averaging all the data points at that latitude (for all longtiude and time values) for those key variables 
#4) Add those anomaly columns to the merged_cckw dataframe 


In [6]:
exp_name = 'TC_3km'
pth = "/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N/%s/"%exp_name
fname = pth+'latlon/diags_gaussian_global_nospinup_r3600x1800.nc'
ds = xr.open_dataset(fname)
ds

<xarray.Dataset> Size: 90GB
Dimensions:   (time: 538, lon: 3600, lat: 552)
Coordinates:
  * time      (time) datetime64[ns] 4kB 2000-03-29T12:00:00 ... 2000-08-10T18...
  * lon       (lon) float64 29kB 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9
  * lat       (lat) float64 4kB -14.95 -14.85 -14.75 ... 39.95 40.05 40.15
Data variables: (12/21)
    rainc     (time, lat, lon) float32 4GB ...
    rainnc    (time, lat, lon) float32 4GB ...
    pr        (time, lat, lon) float32 4GB ...
    prc       (time, lat, lon) float32 4GB ...
    hfx       (time, lat, lon) float32 4GB ...
    mse_vint  (time, lat, lon) float32 4GB ...
    ...        ...
    u850      (time, lat, lon) float32 4GB ...
    ushr      (time, lat, lon) float32 4GB ...
    v10       (time, lat, lon) float32 4GB ...
    v200      (time, lat, lon) float32 4GB ...
    v850      (time, lat, lon) float32 4GB ...
    vshr      (time, lat, lon) float32 4GB ...
Attributes:
    Conventions:  CF-1.7

In [7]:
# -----------------------------
# Step 0. Load your data
# -----------------------------
# Open the input dataset (assumes your infile is in a NetCDF-like format)

# The key variables
key_vars = {'olrtoa', 'mse_vint', 'u200', 'v200', 'u850', 'v850', 'qv_vint'}

# -----------------------------
# Step 1. Identify latitudes of interest
# -----------------------------
# Extract unique latitudes from merged_cckw
lat_interest = np.unique(merged_cckw['lat'])
# Print for debugging:
print("Latitudes of interest:", lat_interest)


Latitudes of interest: [-2.11379   0.900047  1.923851  2.043174  2.728995  2.916088  3.094325
  3.188362  3.221646  3.487211  3.642919  3.665281  3.919714  3.97532
  4.149562  4.405801  4.434918  4.525623  4.622905  4.810503  4.814079
  4.826263  4.925997  5.048761  5.079166  5.175823  5.190067  5.199242
  5.23987   5.258481  5.425529  5.534486  5.626326  5.626872  5.639154
  5.673002  5.709276  5.722951  5.727172  5.762957  5.776806  5.855659
  5.974441  6.019614  6.05166   6.188488  6.224133  6.313917  6.327643
  6.457926  6.479525  6.515817  6.563504  6.587044  6.613551  6.635245
  6.696631  6.720313  6.734026  6.797878  6.839622  6.90836   6.922534
  6.923033  6.936582  6.954613  6.969622  6.985094  7.023653  7.026378
  7.129595  7.30258   7.3837    7.454962  7.495808  7.497995  7.534088
  7.576272  7.5958    7.618085  7.668938  7.717054  7.808681  7.813634
  7.822735  7.905107  7.930477  7.961646  7.971119  7.98055   8.014495
  8.083918  8.213318  8.231096  8.280962  8.28397   8.3

In [8]:
#So this could use some work. First, I should run this for the entire dataset so I have values at all latitudes. Then I need to best map the latitudes of interest to this. 


# # -----------------------------
# # Step 2. Compute background means only for these latitudes
# # -----------------------------
# # Create a dictionary to hold the background (mean) for each variable
# avg_dict = {}
# for var in key_vars:
#     # Compute the mean over time and lon, then select only latitudes of interest
#     avg_mean = ds[var].mean(dim=['time', 'lon'])
#     # Only take values corresponding to latitudes of interest 
#     # (using method='nearest' to match if the latitudes don't exactly coincide)
#     avg_dict[var] = avg_mean.sel(lat=lat_interest, method='nearest')

#SAVE OFF AS NETCDF! 

In [9]:
#load key variables averages dataset generated previously 
avg_ds = xr.open_dataset('lat_avg_values_aquaplanet.nc')

# Step 1: Get the latitude values
lat_vals = avg_ds['lat'].values

# Step 2: Identify unique latitudes and their first index positions
_, unique_indices = np.unique(lat_vals, return_index=True)

# Step 3: Sort those indices to preserve original order
unique_indices_sorted = np.sort(unique_indices)

# Step 4: Select only those unique indices along the lat dimension
avg_ds_non_duplicated = avg_ds.isel(lat=unique_indices_sorted)

avg_ds_non_duplicated

<xarray.Dataset> Size: 53kB
Dimensions:                        (lat: 552)
Coordinates: (12/22)
  * lat                            (lat) float64 4kB -14.95 -14.85 ... 40.15
    rainc                          (lat) float32 2kB ...
    rainnc                         (lat) float32 2kB ...
    pr                             (lat) float32 2kB ...
    prc                            (lat) float32 2kB ...
    hfx                            (lat) float32 2kB ...
    ...                             ...
    u850                           (lat) float32 2kB ...
    ushr                           (lat) float32 2kB ...
    v10                            (lat) float32 2kB ...
    v200                           (lat) float32 2kB ...
    v850                           (lat) float32 2kB ...
    vshr                           (lat) float32 2kB ...
Data variables:
    __xarray_dataarray_variable__  (lat) float32 2kB ...
Attributes:
    __xarray_dataarray_name__:  vshr

In [ ]:

def get_index(coord_array, value):
    """
    Get the index of the closest value in a coordinate array.
    Works for both numeric and datetime values.
    """
    # Convert to NumPy array if it's xarray or pandas
    coord_array = np.asarray(coord_array)
    
    # If datetime, cast both to numpy.datetime64
    if np.issubdtype(coord_array.dtype, np.datetime64):
        value = np.datetime64(value,'ns')
        coord_array = coord_array.astype('datetime64[ns]')
    
    return np.abs(coord_array - value).argmin()



def get_raw_values(row):
    values = {}
    for var in key_vars:
        try:
            #THIS WOULD BE FOR SINGLE POINT 
            # Select the nearest point from ds using the values in the current row
            point_val = ds[var].sel(
                time=row['time'],
                lat=row['lat'],
                lon=row['lon'],
                method='nearest'
            )

            # 1. Get the actual matched coordinate from your sel()
            nearest_time = point_val['time'].item()
            nearest_lat = point_val['lat'].item()
            nearest_lon = point_val['lon'].item()

            lat_idx = get_index(ds['lat'].values, nearest_lat)
            lon_idx = get_index(ds['lon'].values, nearest_lon)
            time_idx = get_index(ds['time'].values, nearest_time)

            
            anomaly_values= avg_ds_non_duplicated[var].isel(lat=slice(lat_idx - half_n, lat_idx + half_n + 1) )

            lat_idx_min = lat_idx - half_n
            lat_idx_max = lat_idx + half_n + 1


            

            #Finding longitude indices a bit more complex b/c we need to deal with exceeding ds lon index range. 
            #Wrap longitude indices in case we exceed 3600 or go below 0. 

            lon_idx_min = lon_idx - half_n
            lon_idx_max = lon_idx + half_n
            
            lon_indices = np.linspace(lon_idx_min,lon_idx_max,(lon_idx_max-lon_idx_min)+1).astype(int)
            fixed_lon_indices = lon_indices % 3600 #deal w/ wrap-around using modulo operator
            
            n_by_n_grid_avg_val = ds[var].isel(
            time=time_idx,
            lat=slice(lat_idx_min, lat_idx_max),
            lon=fixed_lon_indices 
            )
            
            #Old code
            #------------------
            # anomaly_values= avg_ds_non_duplicated[var].isel(lat=slice(lat_idx - half_n, lat_idx + half_n + 1) )
            
            # n_by_n_grid_avg_val = ds[var].isel(
            # time=time_idx,
            # lat=slice(lat_idx - half_n, lat_idx + half_n + 1),
            # lon=slice(lon_idx - half_n, lon_idx + half_n + 1)
            # )
            
            n_by_n_grid_avg_val_anomaly = (n_by_n_grid_avg_val - avg_ds_non_duplicated[var]).mean().item()

            # Save the value with a new column name: var'
            values[var + '_anomaly'] = n_by_n_grid_avg_val_anomaly 
            
        except Exception as e:
            print(f"Error for variable {var} at row {row.name}: {e}")
            values[var] = np.nan  # In case of error, fill with NaN
    return pd.Series(values)

# Apply the function row-wise to create a DataFrame of the raw values
raw_values_df = merged_cckw.apply(get_raw_values, axis=1)
print(raw_values_df)

# Append these new columns to the original merged_cckw DataFrame
merged_cckw_full = pd.concat([merged_cckw, raw_values_df], axis=1)

# Show the first few rows of the updated DataFrame to verify:
print(merged_cckw_full.head())

In [ ]:
#merged_cckw_with_features['olrtoa_anomaly'].hist()

merged_cckw_full['qv_vint_anomaly'].hist()

In [ ]:
# List the key variable column names as features
feature_cols = ['olrtoa_anomaly', 'u200_anomaly', 'v200_anomaly', 'u850_anomaly', 'v850_anomaly', 'vor850','qv_vint_anomaly']
target_col = 'developer'

# Create feature matrix X and target vector y
X = merged_cckw_full[feature_cols]

y = merged_cckw_full[target_col]

#Delete redundant columns if there are redundant columns 
merged_cckw_full = merged_cckw_full.loc[:, ~merged_cckw_full.columns.duplicated()]

print(merged_cckw_full.columns)


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # This gives you a NumPy array

#DOUBLE CHECK THIS IS APPLIED TO EACH COLUMN SEPARATELY 

In [ ]:
# 60-40 split: 60% training and 40% validation (or test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=42, stratify=y)

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns=feature_cols, index=X.index)

In [ ]:
X_scaled.describe()

In [ ]:
model = () 

def set_seeds(seed_value=42):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

# Determine the input dimension (# of features)
input_dim = X_train.shape[1]

set_seeds(42)

model = Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    Dense(8, activation='relu', name='input'),
    Dense(2, activation='relu', name='hidden_layer_3'),
    Dense(1, activation='sigmoid', name='output_layer')  # Sigmoid for binary classification.
])

# Create an optimizer with a custom learning rate
optimizer = Adam(learning_rate=0.0045)   # <-- Set your learning rate here

# Then compile your model using this optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Assuming y_train contains your training labels
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print("Class weights:", class_weight_dict)
#class_weight_dict = {0.0: 0.5, 1.0: 5} #alt weights
set_seeds(42)

ckpt = ModelCheckpoint('best_weights.weights.h5',
                       monitor='val_accuracy',
                       mode='max',
                       save_best_only=True,      # only overwrite when val_loss improves
                       save_weights_only=True)

# Fit the model using class weights
history = model.fit(X_train, y_train,
                    epochs=125,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    class_weight=class_weight_dict,
                    verbose=1,shuffle=True,callbacks=[ckpt]
)

In [ ]:
min_validation_loss = np.min(history.history['val_loss']) 
max_validation_accuracy = np.max(history.history['val_accuracy'])

# --- Plot Accuracy and Loss ---
plt.figure(figsize=(12, 5))

# Accuracy subplot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')

plt.title(f"Model Accuracy {dt_cckw_cross_hrs} hours after CCKW crest cross, "
          f"{half_n*2+1}x{half_n*2+1} grid,\n" 
          f"Model Val Accuracy {max_validation_accuracy:.3f}")


plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss subplot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train  Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title(
    f"Model Loss {dt_cckw_cross_hrs}h after CCKW crest, {half_n*2+1}×{half_n*2+1} grid\n"
    f"Min Validation Loss: {min_validation_loss:.3f}")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()

# make sure your output directory exists
outdir = "ANN_Figures"
os.makedirs(outdir, exist_ok=True)

# construct a filename
fname = f"model_accuracy_and_loss_{dt_cckw_cross_hrs}h_grid{half_n*2+1}.png"

# save the current figure
plt.savefig(
    os.path.join(outdir, fname),
    dpi=300,               # resolution
    bbox_inches="tight"    # trim extra whitespace
)

plt.show()







In [ ]:
#Save off model metrics to csv for later analysis 

# Sample values
grid_size = f"{half_n*2+1}×{half_n*2+1} grid"
time_after_cckw_cross = dt_cckw_cross_hrs
max_accuracy = max_validation_accuracy
min_val_loss = min_validation_loss 

csv_file = "model_metrics.csv"

# Check if the file exists already
file_exists = os.path.isfile(csv_file)

# Open the file in append mode and write headers only if it's new
with open(csv_file, mode="a", newline="") as file:
    writer = csv.writer(file)
    
    # Write header if file doesn't already exist
    if not file_exists:
        writer.writerow(["grid_size", "time_after_cckw_cross", "max_accuracy", "min_val_loss"])
    
    # Write the new row
    writer.writerow([grid_size, time_after_cckw_cross, max_accuracy, min_val_loss])

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.3f}, Test Accuracy: {accuracy:.3f}")

In [ ]:
# Assuming y_test is your test target Series
print("Test set class distribution:")
print(y_test.value_counts())

In [ ]:

#Load weights with best training loss performance 
model.load_weights('best_weights.weights.h5')

# --- Generate Predictions ---
# Predict the probabilities on the test set. 
# Note: For binary classification with a sigmoid output, the output is a probability.
y_pred_probs = model.predict(X_test)

# Convert probabilities to binary predictions using a threshold (default 0.5)
y_pred = (y_pred_probs > 0.5).astype(int)

# If your predictions are in a 2D array (e.g., shape (n, 1)) you might want to flatten them:
y_pred = y_pred.ravel()

# --- Examine Distribution in the Test Set ---
print("Test set class distribution:")
print(y_test.value_counts())

# --- Confusion Matrix ---
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Non-Developer (0)', 'Developer (1)'],
            yticklabels=['Non-Developer (0)', 'Developer (1)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title(f'Confusion Matrix, {dt_cckw_cross_hrs} hours after cckw crest cross,{half_n*2 +1 }x{half_n*2 + 1 } grid')


# make sure your output directory exists
outdir = "ANN_Figures"
os.makedirs(outdir, exist_ok=True)

# construct a filename
fname = f"confusion_matrix_{dt_cckw_cross_hrs}h_grid{half_n*2+1}.png"

# save the current figure
plt.savefig(
    os.path.join(outdir, fname),
    dpi=300,               # resolution
    bbox_inches="tight"    # trim extra whitespace
)

plt.show()

# --- Classification Report ---
report = classification_report(y_test, y_pred, target_names=['Non-Developer', 'Developer'])
print("Classification Report:")
print(report)


# 2) Choose where to save it
outdir = "ANN_Figures"
os.makedirs(outdir, exist_ok=True)
filepath = os.path.join(outdir, f"classification_report_{dt_cckw_cross_hrs}h_grid{half_n*2+1}.txt")

# 3) Write it to disk
with open(filepath, "w") as f:
    f.write("Classification Report:\n")
    f.write(report)

print(f"Saved classification report to {filepath}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=42, stratify=y)

In [ ]:
y_pred_series = pd.Series(
    y_pred,
    index=y_test.index,
    name='y_pred'
)


fp_idx = y_pred_series[(y_pred_series == 1) & (y_test == 0)].index
fn_idx = y_pred_series[(y_pred_series == 0) & (y_test == 1)].index

print(fp_idx)
print(fn_idx)

fp_df = X_test.loc[fp_idx].copy()
fn_df = X_test.loc[fn_idx].copy()
fp_df['error_type'] = 'False Positive'
fn_df['error_type'] = 'False Negative'
fp_df['sample_id']  = fp_df.index.astype(str)   # string so seaborn treats it categorically
fn_df['sample_id']  = fn_df.index.astype(str)

# Combine them
mis_df = pd.concat([fp_df, fn_df], axis=0)



# Melt so you get one row per sample×variable
mis_melted = mis_df.melt(
    id_vars=['error_type','sample_id'],
    value_vars=list(feature_cols),
    var_name='Variable',
    value_name='Anomaly Value'
)

mis_melted

In [ ]:
mis_melted['Scaled'] = (
    mis_melted
      .groupby('Variable')['Anomaly Value']
      .transform(lambda x: (x - x.mean()) / x.std())
)

# First map 0 -> 'Non-Developer', 1 -> 'Developer'
merged_cckw_full['developer_label'] = merged_cckw_full['developer'].map({0: 'Non-Developer', 1: 'Developer'})
# Melt

melted = merged_cckw_full.melt(id_vars='developer_label', value_vars=X_test,var_name='Variable', value_name='Anomaly Value')

# after you’ve created your `melted` DataFrame:

melted['Scaled'] = (
    melted
      .groupby('Variable')['Anomaly Value']
      .transform(lambda x: (x - x.mean()) / x.std())
)



In [ ]:
plt.figure(figsize=(14,9))

# 1) your original boxplot
sns.boxplot(
    data=melted,
    x='Variable',
    y='Scaled',
    hue='developer_label',
    palette='Set2',
    boxprops={'alpha': 0.5}   # <-- make boxes 30% opaque

)

# scatter of all FPs, each with its own hue
fp_only = mis_melted[mis_melted['error_type']=='False Positive']
sns.scatterplot(
    data=fp_only,
    x='Variable', y='Scaled',
    hue='sample_id',         # <–– each sample_id gets its own color
    palette='hsv',           # or 'tab20', 'gist_rainbow', etc.
    marker='o',
    s=100,
    legend=False,            # hide the huge legend, or set True if you want it
    label='False Positive'

)

# scatter of FNs each with its own hue
fn_only = mis_melted[mis_melted['error_type']=='False Negative']
sns.scatterplot(
    data=fn_only,
    x='Variable', y='Scaled',
    hue='sample_id',           # or whatever you like
    marker='X',
    palette='hsv',           # or 'tab20', 'gist_rainbow', etc.
    s=100,
    legend=False,
    label='False Positive'

)


plt.title(f'Anomaly Distribution with Misclassifications Overlay, Evaluated {dt_cckw_cross_hrs} hours after cckw crest cross, {half_n*2 +1 }x{half_n*2 + 1 } grid')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# make sure your output directory exists
outdir = "ANN_Figures"
os.makedirs(outdir, exist_ok=True)

# construct a filename
fname = f"anomaly distribution {dt_cckw_cross_hrs}h_grid{half_n*2+1}.png"

# save the current figure
plt.savefig(
    os.path.join(outdir, fname),
    dpi=300,               # resolution
    bbox_inches="tight"    # trim extra whitespace
)

plt.show()
